In [392]:
# Sometimes, part of your data appears to be numbers but they are actually strings. Before you train your model, first use df.hist() - This will plot the histogram of all numerical columns.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [393]:
df = pd.read_csv("./data/laptop_price.csv", encoding='latin-1')
pd.set_option("display.max_column", 50)
pd.set_option("display.max_row", 6)
df
# The default utf-8 was not able to encode the data for some reasons. That's why I specified an encoding format of `latin-14`

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00
1302,1320,Asus,X553SA-XX031T (N3050/4GB/500GB/W10),Notebook,15.6,1366x768,Intel Celeron Dual Core N3050 1.6GHz,4GB,500GB HDD,Intel HD Graphics,Windows 10,2.2kg,369.00


In [394]:
df = df.drop(['Product'], axis=1)

In [395]:
df = df.join(pd.get_dummies(df["Company"]).astype("int"))
df = df.drop(['Company'], axis=1)

In [396]:
df = df.join(pd.get_dummies(df["TypeName"]).astype("int"))
df = df.drop(['TypeName'], axis=1)

In [397]:
Screen_Width_Height = df["ScreenResolution"].apply(lambda X: X.split(" ")[-1])
df["Screen_Width"] = Screen_Width_Height.apply(lambda X: X.split("x")[0]).astype("int")
df["Screen_Height"] = Screen_Width_Height.apply(lambda X: X.split("x")[1]).astype("int")
df = df.drop(["ScreenResolution"], axis=1)

In [398]:
df["CPU_Brand"] = df["Cpu"].apply(lambda X: X.split(" ")[0])
df["CPU_Freq"] = df["Cpu"].apply(lambda X: X.split(" ")[-1][:-3]).astype("float")
CPU_df = pd.get_dummies(df["CPU_Brand"])
CPU_df.columns = [col + "_CPU" for col in CPU_df.columns]
df = df.join(CPU_df)
df = df.drop(["Cpu" ,"CPU_Brand"], axis=1)

In [399]:
df

,laptop_ID,Inches,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU
0,1,13.3,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2560,1600,2.3,0,1,0
1,2,13.3,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1440,900,1.8,0,1,0
2,3,15.6,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1920,1080,2.5,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,14.0,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0
1301,1319,15.6,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,2.5,0,1,0
1302,1320,15.6,4GB,500GB HDD,Intel HD Graphics,Windows 10,2.2kg,369.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0


In [400]:
def to_mb(value):
    if "GB" in value:
        return float(value[:value.find("GB")]) * 1000
    elif "TB" in value:
        return float(value[:value.find("TB")]) * 1000000

In [401]:
df["RAM(mb)"] = df["Ram"].apply(to_mb)
df = df.drop(["Ram"], axis=1)

In [402]:
df["Memory_size"] = df["Memory"].apply(lambda X: X.split(" ")[0][:-2]).astype("float")
df["Storage_type"] = df["Memory"].apply(lambda X: X.split(" ")[1])
df = df.join(pd.get_dummies(df["Storage_type"]))
df = df.drop(["Memory"], axis=1)
df = df.drop(["Storage_type"], axis=1)
df

,laptop_ID,Inches,Gpu,OpSys,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU,RAM(mb),Memory_size,Flash,HDD,Hybrid,SSD
0,1,13.3,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2560,1600,2.3,0,1,0,8000.0,128.0,0,0,0,1
1,2,13.3,Intel HD Graphics 6000,macOS,1.34kg,898.94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1440,900,1.8,0,1,0,8000.0,128.0,1,0,0,0
2,3,15.6,Intel HD Graphics 620,No OS,1.86kg,575.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1920,1080,2.5,0,1,0,8000.0,256.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,14.0,Intel HD Graphics,Windows 10,1.5kg,229.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0,2000.0,64.0,1,0,0,0
1301,1319,15.6,AMD Radeon R5 M330,Windows 10,2.19kg,764.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,2.5,0,1,0,6000.0,1.0,0,1,0,0
1302,1320,15.6,Intel HD Graphics,Windows 10,2.2kg,369.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0,4000.0,500.0,0,1,0,0


In [403]:
df["GPU_Brand"] = df["Gpu"].apply(lambda X: X.split(" ")[0])
GPU_Brands_df = pd.get_dummies(df["GPU_Brand"])
GPU_Brands_df.columns = [col + "_GPU" for col in GPU_Brands_df.columns]
df = df.join(GPU_Brands_df)
df = df.drop(["Gpu", "GPU_Brand"], axis=1)
df

,laptop_ID,Inches,OpSys,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU,RAM(mb),Memory_size,Flash,HDD,Hybrid,SSD,AMD_GPU,ARM_GPU,Intel_GPU,Nvidia_GPU
0,1,13.3,macOS,1.37kg,1339.69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2560,1600,2.3,0,1,0,8000.0,128.0,0,0,0,1,0,0,1,0
1,2,13.3,macOS,1.34kg,898.94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1440,900,1.8,0,1,0,8000.0,128.0,1,0,0,0,0,0,1,0
2,3,15.6,No OS,1.86kg,575.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1920,1080,2.5,0,1,0,8000.0,256.0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,14.0,Windows 10,1.5kg,229.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0,2000.0,64.0,1,0,0,0,0,0,1,0
1301,1319,15.6,Windows 10,2.19kg,764.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,2.5,0,1,0,6000.0,1.0,0,1,0,0,1,0,0,0
1302,1320,15.6,Windows 10,2.2kg,369.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1366,768,1.6,0,1,0,4000.0,500.0,0,1,0,0,0,0,1,0


In [404]:
op_sys = pd.get_dummies(df["OpSys"])
df = df.join(op_sys)
df = df.drop(["OpSys"], axis=1)
df

,laptop_ID,Inches,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,...,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU,RAM(mb),Memory_size,Flash,HDD,Hybrid,SSD,AMD_GPU,ARM_GPU,Intel_GPU,Nvidia_GPU,Android,Chrome OS,Linux,Mac OS X,No OS,Windows 10,Windows 10 S,Windows 7,macOS
0,1,13.3,1.37kg,1339.69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2560,1600,2.3,0,1,0,8000.0,128.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
1,2,13.3,1.34kg,898.94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1440,900,1.8,0,1,0,8000.0,128.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,3,15.6,1.86kg,575.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1920,1080,2.5,0,1,0,8000.0,256.0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,14.0,1.5kg,229.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,1366,768,1.6,0,1,0,2000.0,64.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1301,1319,15.6,2.19kg,764.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1366,768,2.5,0,1,0,6000.0,1.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1302,1320,15.6,2.2kg,369.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1366,768,1.6,0,1,0,4000.0,500.0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [405]:
df["Weight"] = df["Weight"].apply(lambda X: X[:-2]).astype("float")
df

,laptop_ID,Inches,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,...,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU,RAM(mb),Memory_size,Flash,HDD,Hybrid,SSD,AMD_GPU,ARM_GPU,Intel_GPU,Nvidia_GPU,Android,Chrome OS,Linux,Mac OS X,No OS,Windows 10,Windows 10 S,Windows 7,macOS
0,1,13.3,1.37,1339.69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2560,1600,2.3,0,1,0,8000.0,128.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
1,2,13.3,1.34,898.94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1440,900,1.8,0,1,0,8000.0,128.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,3,15.6,1.86,575.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1920,1080,2.5,0,1,0,8000.0,256.0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1318,14.0,1.50,229.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,1366,768,1.6,0,1,0,2000.0,64.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1301,1319,15.6,2.19,764.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1366,768,2.5,0,1,0,6000.0,1.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1302,1320,15.6,2.20,369.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1366,768,1.6,0,1,0,4000.0,500.0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [413]:
# plt.figure(figsize=(10, 5))
pd.set_option("de")
df.corr()

,laptop_ID,Inches,Weight,Price_euros,Acer,Apple,Asus,Chuwi,Dell,Fujitsu,Google,HP,Huawei,LG,Lenovo,MSI,Mediacom,Microsoft,Razer,Samsung,Toshiba,Vero,Xiaomi,2 in 1 Convertible,Gaming,...,Screen_Width,Screen_Height,CPU_Freq,AMD_CPU,Intel_CPU,Samsung_CPU,RAM(mb),Memory_size,Flash,HDD,Hybrid,SSD,AMD_GPU,ARM_GPU,Intel_GPU,Nvidia_GPU,Android,Chrome OS,Linux,Mac OS X,No OS,Windows 10,Windows 10 S,Windows 7,macOS
laptop_ID,1.000000,-0.087796,-0.011798,0.067830,-0.057721,-0.119585,-0.044252,-0.043348,-0.013628,0.009268,-0.012039,0.059002,-0.047711,0.022884,0.031822,0.054900,-0.015268,-0.051140,0.029371,0.029782,0.031935,0.019820,-0.030861,0.066227,0.032384,...,-0.042878,-0.053912,0.102716,-0.055338,0.049765,0.039920,-0.028607,0.052683,0.028792,0.050182,0.098241,-0.079509,-0.089050,0.039920,0.094479,-0.037596,-0.007219,0.046729,-0.015589,0.009431,-0.069617,-0.008729,-0.061942,0.194094,-0.158931
Inches,-0.087796,1.000000,0.827631,0.068197,0.009435,-0.158850,0.112766,-0.017426,0.045288,0.019637,-0.091551,-0.026828,-0.055473,0.001667,-0.055055,0.207877,-0.080415,-0.072377,-0.025923,-0.069898,-0.090946,-0.046401,-0.022075,-0.314691,0.402666,...,-0.071245,-0.095404,0.307870,0.129801,-0.122000,-0.052816,0.237993,-0.098539,-0.367908,0.271145,0.026080,-0.082325,0.207023,-0.052816,-0.592264,0.486561,-0.135222,-0.236792,0.094654,-0.127740,0.110131,0.088614,-0.093285,-0.064775,-0.100867
Weight,-0.011798,0.827631,1.000000,0.210370,0.056444,-0.138191,0.101332,-0.022536,0.088465,0.014053,-0.067790,-0.098842,-0.058276,-0.071401,-0.029767,0.194364,-0.077511,-0.080508,0.017343,-0.090664,-0.117163,-0.059121,-0.034930,-0.236693,0.591756,...,-0.032880,-0.053846,0.320434,0.062641,-0.057386,-0.037026,0.383874,-0.105692,-0.259337,0.166644,0.030970,-0.037134,0.106519,-0.037026,-0.596378,0.565232,-0.079495,-0.139835,0.062424,-0.100576,0.051578,0.082438,-0.087904,-0.048100,-0.096042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Windows 10 S,-0.061942,-0.093285,-0.087904,0.018312,-0.023027,-0.010059,0.030999,-0.003776,-0.042706,-0.003776,-0.003776,-0.040558,-0.003082,-0.003776,-0.042706,-0.016343,-0.005776,0.865357,-0.005776,-0.006555,-0.015371,-0.004361,-0.004361,-0.025147,-0.033961,...,0.022050,0.068859,-0.015338,0.028583,-0.028086,-0.002178,-0.021374,-0.008289,0.022759,-0.049963,-0.006912,0.037502,-0.002994,-0.002178,0.050741,-0.052311,-0.003082,-0.011433,-0.017568,-0.006178,-0.018155,-0.169317,1.000000,-0.014865,-0.007890
Windows 7,0.194094,-0.064775,-0.048100,0.152381,-0.055411,-0.024206,-0.070257,-0.009086,-0.032633,-0.009086,-0.009086,0.139619,-0.007416,-0.009086,0.027481,-0.039326,-0.013900,-0.012864,-0.013900,-0.015773,0.007637,-0.010495,-0.010495,-0.046032,-0.081723,...,0.010784,0.007162,0.052773,-0.042274,0.042631,-0.005242,-0.017529,0.080875,-0.046741,-0.055245,0.127851,0.051767,-0.026998,-0.005242,0.059740,-0.043869,-0.007416,-0.027512,-0.042274,-0.014865,-0.043687,-0.407434,-0.014865,1.000000,-0.018986
macOS,-0.158931,-0.100867,-0.096042,0.089928,-0.029411,0.784352,-0.037291,-0.004822,-0.054545,-0.004822,-0.004822,-0.051802,-0.003936,-0.004822,-0.054545,-0.020873,-0.007378,-0.006828,-0.007378,-0.008372,-0.019632,-0.005571,-0.005571,-0.032119,-0.043376,...,0.107070,0.156369,0.004821,-0.022438,0.022627,-0.002782,0.028914,0.062323,0.041501,-0.063814,-0.008828,0.041839,0.026948,-0.002782,0.043447,-0.066813,-0.003936,-0.014603,-0.022438,-0.007890,-0.023188,-0.216256,-0.007890,-0.018986,1.000000
